In [88]:
import os
import re
import sklearn
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *

pd.set_option('display.max_columns', None)

In [89]:
DATA_PATH_M = r'C:/Users/FLUXNATURE/Desktop/New Kaggle world/NCAAM_satge_two/MDataFiles_Stage2/'

for filename in os.listdir(DATA_PATH_M):
    print(filename)

Cities.csv
Conferences.csv
MConferenceTourneyGames.csv
MGameCities.csv
MMasseyOrdinals.csv
MNCAATourneyCompactResults.csv
MNCAATourneyDetailedResults.csv
MNCAATourneySeedRoundSlots.csv
MNCAATourneySeeds.csv
MNCAATourneySlots.csv
MRegularSeasonCompactResults.csv
MRegularSeasonDetailedResults.csv
MSampleSubmissionStage2.csv
MSeasons.csv
MSecondaryTourneyCompactResults.csv
MSecondaryTourneyTeams.csv
MTeamCoaches.csv
MTeamConferences.csv
MTeams.csv
MTeamSpellings.csv


In [90]:
tourney_compact_df = pd.read_csv(DATA_PATH_M + 'MNCAATourneyCompactResults.csv')
tourney_compact_df['is_tourney'] = 1.0

In [91]:
regseason_compact_df = pd.read_csv(DATA_PATH_M + 'MRegularSeasonCompactResults.csv')
regseason_compact_df['is_tourney'] = 0.0

In [92]:
all_games_df = regseason_compact_df.append(tourney_compact_df)
all_games_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,is_tourney
0,1985,20,1228,81,1328,64,N,0,0.0
1,1985,25,1106,77,1354,70,H,0,0.0
2,1985,25,1112,63,1223,56,H,0,0.0
3,1985,25,1165,70,1432,54,H,0,0.0
4,1985,25,1192,86,1447,74,H,0,0.0


In [93]:
all_games_df.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,is_tourney
2246,2019,146,1120,77,1246,71,N,1,1.0
2247,2019,146,1277,68,1181,67,N,0,1.0
2248,2019,152,1403,61,1277,51,N,0,1.0
2249,2019,152,1438,63,1120,62,N,0,1.0
2250,2019,154,1438,85,1403,77,N,1,1.0


Merge spread, seeds, ranks, and Ken POM data

Load and merge on point spread

In [94]:
spread_df = pd.read_csv('spreads_all.csv')

C:\Users\FLUXNATURE\Anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (2,4,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [95]:
spread_df = spread_df[['Season','date','line','lineavg','road','home','rscore','hscore','WScore','LScore','HTeamID','RTeamID','WTeamID','LTeamID']]

Merge spread_df and all_games_df

In [96]:
all_games_df = pd.merge(all_games_df, spread_df, left_on=['WTeamID','LTeamID','Season','WScore','LScore'], right_on = ['WTeamID','LTeamID','Season','WScore','LScore'], how='inner')

C:\Users\FLUXNATURE\Anaconda3\envs\tf\lib\site-packages\pandas\core\reshape\merge.py:1108: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,


In [97]:
all_games_df = all_games_df[['Season','date','is_tourney','home','HTeamID','hscore','road','RTeamID','rscore','line','lineavg']]

In [98]:
all_games_df['Hwin'] = 0
all_games_df.loc[all_games_df['hscore']>all_games_df['rscore'], 'Hwin'] = 1

In [99]:
all_games_df.head()

,Season,date,is_tourney,home,HTeamID,hscore,road,RTeamID,rscore,line,lineavg,Hwin
0,2004,11/24/2003,0.0,arizona st.,1113.0,83,cal st. fullerton,1168.0,76,19.0,23.20,1
1,2004,11/24/2003,0.0,dayton,1173.0,82,central michigan,1141.0,63,11.0,5.88,1
2,2004,11/24/2003,0.0,detroit,1178.0,64,western michigan,1444.0,43,1.0,5.63,1
3,2004,11/24/2003,0.0,st. john's,1385.0,59,fairfield,1193.0,64,12.5,13.81,0
4,2004,11/24/2003,0.0,gonzaga,1211.0,90,denver,1176.0,58,19.5,18.45,1


In [100]:
all_games_df.tail()

,Season,date,is_tourney,home,HTeamID,hscore,road,RTeamID,rscore,line,lineavg,Hwin
65137,2019,03/29/2019,1.0,michigan st.,1277.0,80,lsu,1261.0,63,6.0,6.43,1
65138,2019,03/30/2019,1.0,gonzaga,1211.0,69,texas tech,1403.0,75,4.5,3.80,0
65139,2019,03/30/2019,1.0,virginia,1438.0,80,purdue,1345.0,75,4.5,3.77,1
65140,2019,03/31/2019,1.0,kentucky,1246.0,71,auburn,1120.0,77,4.5,1.53,0
65141,2019,03/31/2019,1.0,duke,1181.0,67,michigan st.,1277.0,68,2.0,1.25,0


Include seed

In [101]:
seeds_df = pd.read_csv(DATA_PATH_M + 'MNCAATourneySeeds.csv')
seeds_df['Seed_num'] = seeds_df['Seed'].str.extract('(\d\d)', expand=True)
seeds_df['Seed_num'] = pd.to_numeric(seeds_df['Seed_num'])
seeds_df = seeds_df[['Season','TeamID','Seed_num']]
seeds_df.rename(columns={'Seed_num': 'Seed'}, inplace=True)
seeds_df.head()

,Season,TeamID,Seed
0,1985,1207,1
1,1985,1210,2
2,1985,1228,3
3,1985,1260,4
4,1985,1374,5


In [102]:
#Note: we merge twice for this merge and subsequent merges, because the data are in a wide format by team (i.e., there are two variables rather than two rows for each game)
home_road = ['H','R']
for hr in home_road:
    all_games_df = pd.merge(all_games_df, seeds_df, left_on=[hr+'TeamID','Season'], right_on = ['TeamID','Season'], how='left')
    all_games_df.rename(columns={'Seed': hr+'Seed'}, inplace=True)
    all_games_df = all_games_df.drop(['TeamID'], axis=1)

In [103]:
all_games_df.head()

,Season,date,is_tourney,home,HTeamID,hscore,road,RTeamID,rscore,line,lineavg,Hwin,HSeed,RSeed
0,2004,11/24/2003,0.0,arizona st.,1113.0,83,cal st. fullerton,1168.0,76,19.0,23.20,1,NaN,NaN
1,2004,11/24/2003,0.0,dayton,1173.0,82,central michigan,1141.0,63,11.0,5.88,1,10.0,NaN
2,2004,11/24/2003,0.0,detroit,1178.0,64,western michigan,1444.0,43,1.0,5.63,1,NaN,11.0
3,2004,11/24/2003,0.0,st. john's,1385.0,59,fairfield,1193.0,64,12.5,13.81,0,NaN,NaN
4,2004,11/24/2003,0.0,gonzaga,1211.0,90,denver,1176.0,58,19.5,18.45,1,2.0,NaN


Merge Ken Pom data and all_games_df

In [104]:
kp_df = pd.read_csv('kp_all.csv')

In [105]:
efficiency_list = ['conf','adjem','adjo','adjd','luck']
for hr in home_road:
    all_games_df = pd.merge(all_games_df, kp_df, left_on=[hr+'TeamID','Season'], right_on = ['TeamID','Season'], how='inner')
    for metric in efficiency_list:
        all_games_df.rename(columns={metric: hr+metric}, inplace=True)
    all_games_df = all_games_df.drop(['TeamID','team'], axis=1)

In [106]:
all_games_df.head()

,Season,date,is_tourney,home,HTeamID,hscore,road,RTeamID,rscore,line,lineavg,Hwin,HSeed,RSeed,Hconf,Hadjem,Hadjo,Hadjd,Hluck,Rconf,Radjem,Radjo,Radjd,Rluck
0,2004,11/24/2003,0.0,arizona st.,1113.0,83,cal st. fullerton,1168.0,76,19.0,23.20,1,NaN,NaN,P10,1.29,107.6,106.3,-0.049,BW,-4.85,102.9,107.8,-0.023
1,2004,12/01/2003,0.0,pepperdine,1337.0,74,cal st. fullerton,1168.0,78,17.0,19.35,0,NaN,NaN,WCC,2.93,113.7,110.8,-0.001,BW,-4.85,102.9,107.8,-0.023
2,2004,01/17/2004,0.0,uc irvine,1414.0,74,cal st. fullerton,1168.0,67,7.0,7.17,1,NaN,NaN,BW,-4.79,101.4,106.2,0.034,BW,-4.85,102.9,107.8,-0.023
3,2004,01/29/2004,0.0,cal poly slo,1142.0,80,cal st. fullerton,1168.0,71,5.5,4.70,1,NaN,NaN,BW,-4.69,101.1,105.8,-0.015,BW,-4.85,102.9,107.8,-0.023
4,2004,02/21/2004,0.0,utah st.,1429.0,87,cal st. fullerton,1168.0,61,17.0,18.80,1,NaN,NaN,BW,13.60,115.9,102.3,0.037,BW,-4.85,102.9,107.8,-0.023


Add Massey and Ken Pom rankings

In [107]:
massey_df = pd.read_csv(DATA_PATH_M + 'MMasseyOrdinals.csv')
massey_df19 = massey_df.loc[massey_df['Season'] == 2021]

#Ranking at the end of season versus end of tourney (as shown in KP data)
POM_df_else = massey_df[massey_df['SystemName'].str.contains("POM")]
POM_end_df_else = POM_df_else.loc[POM_df_else['RankingDayNum'] == 133]
POM_df19 = massey_df19[massey_df19['SystemName'].str.contains("POM")]
POM_end_df_19 = POM_df19.loc[POM_df19['RankingDayNum'] == 128]
POM_end_df = POM_end_df_else.append(POM_end_df_19)
POM_end_df = POM_end_df[['Season','TeamID','OrdinalRank']]
POM_end_df.rename(columns={'OrdinalRank': 'RankPOM'}, inplace=True)

In [108]:
POM_end_df.tail()

,Season,TeamID,RankPOM
4317029,2021,1467,273
4317030,2021,1468,177
4317031,2021,1469,323
4317032,2021,1470,262
4317033,2021,1471,231


In [109]:
for hr in home_road:
    all_games_df = pd.merge(all_games_df,POM_end_df, left_on=[hr+'TeamID','Season'], right_on = ['TeamID','Season'], how='left')
    all_games_df.rename(columns={'RankPOM': hr+'RankPOM'}, inplace=True)
    all_games_df = all_games_df.drop(['TeamID'], axis=1)

Calculate regular season avg stats

In [110]:
regseason_detail_df = pd.read_csv(DATA_PATH_M + 'MRegularSeasonDetailedResults.csv')

In [111]:
regseason_detail_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [112]:
#Attach the POM data
teamslist = ['W','L']
for wl in teamslist:
    regseason_detail_df = pd.merge(regseason_detail_df, POM_end_df, left_on=[wl+'TeamID','Season'], right_on = ['TeamID','Season'], how='left')
    regseason_detail_df.rename(columns={'RankPOM': wl+'RankPOM'}, inplace=True)
    regseason_detail_df = regseason_detail_df.drop(['TeamID'], axis=1)

Regular season (create one row for each team with avg stats and win info)

In [113]:
win_team_only = regseason_detail_df.drop(['LTeamID','WAst','LAst','LStl','WStl','LBlk','WBlk','LPF','WPF','NumOT','DayNum'], axis=1)
win_team_only.rename(columns={'WTeamName': 'TeamName'}, inplace=True)
win_team_only.rename(columns={'WTeamID': 'TeamID'}, inplace=True)

team_factors_list = ['Score','FGM','FGA','FGA3','FGM3','TO','FTM','FTA','RankPOM']
for factor in team_factors_list:
    win_team_only.rename(columns={'W'+factor: factor}, inplace=True)
    win_team_only.rename(columns={'L'+factor: 'Opponent'+factor}, inplace=True)

win_team_only['wins'] = 1 

win_team_only['wins_top25'] = 0
win_team_only.loc[win_team_only['OpponentRankPOM']<26, 'wins_top25'] = 1

win_team_only['wins_top5'] = 0
win_team_only.loc[win_team_only['OpponentRankPOM']<6, 'wins_top5'] = 1

win_team_only = win_team_only[['Season','TeamID','wins','wins_top25','wins_top5','Score','OpponentScore','FGM','FGA','OpponentFGM','OpponentFGA','FGM3','FGA3','OpponentFGM3','OpponentFGA3','FTM','FTA','OpponentFTM','OpponentFTA','TO','OpponentTO']]

In [114]:
loss_team_only = regseason_detail_df.drop(['WTeamID','WAst','LAst','LStl','WStl','LBlk','WBlk','LPF','WPF','NumOT','DayNum'], axis=1)
loss_team_only.rename(columns={'LTeamName': 'TeamName'}, inplace=True)
loss_team_only.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

for factor in team_factors_list:
    loss_team_only.rename(columns={'L'+factor: factor}, inplace=True)
    loss_team_only.rename(columns={'W'+factor: 'Opponent'+factor}, inplace=True)

loss_team_only['wins'] = 0
loss_team_only['wins_top25'] = 0
loss_team_only['wins_top5'] = 0

loss_team_only = loss_team_only[['Season','TeamID','wins','wins_top25','wins_top5','Score','OpponentScore','FGM','FGA','OpponentFGM','OpponentFGA','FGM3','FGA3','OpponentFGM3','OpponentFGA3','FTM','FTA','OpponentFTM','OpponentFTA','TO','OpponentTO']]

In [115]:
reg_season_all = win_team_only.append(loss_team_only)

In [116]:
reg_season_all['TOmargin'] = reg_season_all['TO']-reg_season_all['OpponentTO']

reg_season_all['PointMargin'] = reg_season_all['Score']-reg_season_all['OpponentScore']

reg_season_all['FG'] = reg_season_all['FGM']/reg_season_all['FGA']
reg_season_all['FGopponent'] = reg_season_all['OpponentFGM']/reg_season_all['OpponentFGA']

reg_season_all['FG3'] = reg_season_all['FGM3']/reg_season_all['FGA3']
reg_season_all['FG3opponent'] = reg_season_all['OpponentFGM3']/reg_season_all['OpponentFGA3']

reg_season_all['FT'] = reg_season_all['FTM']/reg_season_all['FTA']

In [117]:
reg_season_all = reg_season_all.drop(['TO','OpponentTO','FGM','FGA','OpponentFGA','OpponentFGM','FGM3','FGA3','OpponentFGM3','OpponentFGA3','FTM','FTA','OpponentFTM','OpponentFTA'], axis=1)

In [118]:
list(reg_season_all)

['Season',
 'TeamID',
 'wins',
 'wins_top25',
 'wins_top5',
 'Score',
 'OpponentScore',
 'TOmargin',
 'PointMargin',
 'FG',
 'FGopponent',
 'FG3',
 'FG3opponent',
 'FT']

Collapse regular season sums/means

In [119]:
reg_season_means = reg_season_all.groupby(['TeamID','Season']).mean().reset_index()

In [120]:
reg_season_means = reg_season_means.drop(['wins','wins_top25','wins_top5'], axis=1)

In [121]:
reg_season_sum = reg_season_all.groupby(['TeamID','Season']).sum().reset_index()

In [122]:
reg_season_sum = reg_season_sum.drop(['Score','OpponentScore','TOmargin','PointMargin','FG','FG3','FGopponent','FG3opponent','FT'], axis=1)

In [123]:
regseason_df = pd.merge(reg_season_means, reg_season_sum, left_on=['TeamID','Season'], right_on = ['TeamID','Season'], how='left')

In [124]:
regseason_df.head()

,TeamID,Season,Score,OpponentScore,TOmargin,PointMargin,FG,FGopponent,FG3,FG3opponent,FT,wins,wins_top25,wins_top5
0,1101,2014,63.142857,78.619048,2.857143,-15.476190,0.410184,0.510776,0.380275,0.369975,0.758031,2,0,0
1,1101,2015,61.000000,71.857143,-0.642857,-10.857143,0.407501,0.493864,0.381507,0.372684,0.736328,7,0,0
2,1101,2016,69.851852,76.259259,-0.037037,-6.407407,0.442606,0.493308,0.364517,0.366441,0.701402,9,0,0
3,1101,2017,67.880000,72.640000,-0.320000,-4.760000,0.457867,0.463728,0.371026,0.354196,0.640438,9,0,0
4,1101,2018,70.666667,72.814815,-1.074074,-2.148148,0.452366,0.458319,0.327240,0.316082,0.705605,12,0,0


In [125]:
regseason_df.tail()

,TeamID,Season,Score,OpponentScore,TOmargin,PointMargin,FG,FGopponent,FG3,FG3opponent,FT,wins,wins_top25,wins_top5
6529,1467,2021,66.277778,66.666667,-1.388889,-0.388889,0.448842,0.420911,0.313038,0.337331,0.643874,36,0,0
6530,1468,2021,72.555556,68.277778,-4.222222,4.277778,0.500646,0.461030,0.366353,0.391854,0.742075,44,0,0
6531,1469,2021,67.631579,78.210526,1.578947,-10.578947,0.403965,0.460310,0.285279,0.317359,0.730011,24,0,0
6532,1470,2021,63.866667,68.133333,-3.800000,-4.266667,0.432487,0.436650,0.361324,0.369470,0.741663,20,0,0
6533,1471,2021,68.642857,76.571429,1.000000,-7.928571,0.459440,0.482136,0.372476,0.368736,0.604140,16,0,0


In [126]:
regseason_df.to_csv('regseason_df.csv', index=False)

Merge regular season stats with game level data

In [127]:
#Merge the avg regular season performance with all games and have performance wide: 
performance_list = ['wins','wins_top5','wins_top25','Score','OpponentScore','TOmargin','PointMargin','FG','FGopponent','FG3','FG3opponent','FT','RBMargin']
for hr in home_road:
    all_games_df = pd.merge(all_games_df, regseason_df, left_on=[hr+'TeamID','Season'], right_on = ['TeamID','Season'], how='left')
    for var in performance_list:
        all_games_df.rename(columns={var: hr+var}, inplace=True)
    all_games_df = all_games_df.drop(['TeamID'], axis=1)

Add binary experience dummy

In [128]:
temp_df = all_games_df[all_games_df['is_tourney'] == 1]
temp_df_home = temp_df[['home']]
temp_df_home.rename(columns={'home': 'team'}, inplace=True)
temp_df_road = temp_df[['road']]
temp_df_road.rename(columns={'road': 'team'}, inplace=True)
tourney_teams_df = temp_df_home.append(temp_df_road)
tourney_teams_df['team'].value_counts(dropna=False)

C:\Users\FLUXNATURE\Anaconda3\envs\tf\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


kansas              51
north carolina      50
kentucky            48
duke                47
michigan st.        46
                    ..
jacksonville st.     1
coastal carolina     1
detroit              1
colgate              1
prairie view         1
Name: team, Length: 235, dtype: int64

In [129]:
all_games_df['Htourny20plus'] = 0
all_games_df['Rtourny20plus'] = 0

experienced_teams = ['kansas','north carolina','kentucky','duke','michigan st.','wisconsin','florida','villanova','gonzaga','louisville','arizona','xavier','connecticut','syracuse','butler','ohio st.','ucla','west virginia','texas','michigan','pittsburgh','memphis','oregon']
for team in experienced_teams:
    all_games_df.loc[all_games_df['home']==team, 'Htourny20plus'] = 1
    all_games_df.loc[all_games_df['road']==team, 'Rtourny20plus'] = 1

In [130]:
temp_df = all_games_df[all_games_df['is_tourney'] == 1]
temp_df_home = temp_df[['Hconf']]
temp_df_home.rename(columns={'Hconf': 'conf'}, inplace=True)
temp_df_road = temp_df[['Rconf']]
temp_df_road.rename(columns={'Rconf': 'conf'}, inplace=True)
tourny_teams_df = temp_df_home.append(temp_df_road)
tourny_teams_df['conf'].value_counts(dropna=False)

ACC     235
BE      234
B10     223
B12     202
SEC     187
A10      87
P10      73
P12      73
MWC      57
WCC      56
CUSA     53
MVC      52
Amer     35
CAA      33
Horz     32
WAC      24
BW       23
OVC      23
MAC      21
Ivy      20
MAAC     20
SC       20
ASun     20
Pat      19
BSky     17
MEAC     17
AE       16
BSth     16
NEC      16
Slnd     15
SB       15
Sum      14
SWAC     11
MCon      3
Name: conf, dtype: int64

In [131]:
all_games_df['HBig4Conf'] = 0
all_games_df['RBig4Conf'] = 0
conferences = ['ACC','B10','B12','SEC', 'BE']
for conf in conferences:
    all_games_df.loc[all_games_df['Hconf']==conf, 'HBig4Conf'] = 1
    all_games_df.loc[all_games_df['Rconf']==conf, 'RBig4Conf'] = 1

In [132]:
all_games_df.tail()

,Season,date,is_tourney,home,HTeamID,hscore,road,RTeamID,rscore,line,lineavg,Hwin,HSeed,RSeed,Hconf,Hadjem,Hadjo,Hadjd,Hluck,Rconf,Radjem,Radjo,Radjd,Rluck,HRankPOM,RRankPOM,HScore,HOpponentScore,HTOmargin,HPointMargin,HFG,HFGopponent,HFG3,HFG3opponent,HFT,Hwins,Hwins_top25,Hwins_top5,RScore,ROpponentScore,RTOmargin,RPointMargin,RFG,RFGopponent,RFG3,RFG3opponent,RFT,Rwins,Rwins_top25,Rwins_top5,Htourny20plus,Rtourny20plus,HBig4Conf,RBig4Conf
71322,2021,02/07/2021,0.0,loyola-maryland,1259.0,75,lafayette,1248.0,62,2.0,-0.91,1,NaN,NaN,Pat,-4.34,97.0,101.3,-0.19,Pat,-8.44,103.1,111.5,0.062,214.0,253.0,72.117647,70.941176,1.176471,1.176471,0.464743,0.429341,0.303375,0.339861,0.607424,24,0,0,77.666667,76.666667,-0.933333,1.0,0.437864,0.459527,0.357791,0.347018,0.815865,36,0,0,0,0,0,0
71323,2021,02/14/2021,0.0,loyola-maryland,1259.0,88,lafayette,1248.0,69,3.0,1.06,1,NaN,NaN,Pat,-4.34,97.0,101.3,-0.19,Pat,-8.44,103.1,111.5,0.062,207.0,251.0,72.117647,70.941176,1.176471,1.176471,0.464743,0.429341,0.303375,0.339861,0.607424,24,0,0,77.666667,76.666667,-0.933333,1.0,0.437864,0.459527,0.357791,0.347018,0.815865,36,0,0,0,0,0,0
71324,2021,02/14/2021,0.0,loyola-maryland,1259.0,88,lafayette,1248.0,69,3.0,1.06,1,NaN,NaN,Pat,-4.34,97.0,101.3,-0.19,Pat,-8.44,103.1,111.5,0.062,207.0,253.0,72.117647,70.941176,1.176471,1.176471,0.464743,0.429341,0.303375,0.339861,0.607424,24,0,0,77.666667,76.666667,-0.933333,1.0,0.437864,0.459527,0.357791,0.347018,0.815865,36,0,0,0,0,0,0
71325,2021,02/14/2021,0.0,loyola-maryland,1259.0,88,lafayette,1248.0,69,3.0,1.06,1,NaN,NaN,Pat,-4.34,97.0,101.3,-0.19,Pat,-8.44,103.1,111.5,0.062,214.0,251.0,72.117647,70.941176,1.176471,1.176471,0.464743,0.429341,0.303375,0.339861,0.607424,24,0,0,77.666667,76.666667,-0.933333,1.0,0.437864,0.459527,0.357791,0.347018,0.815865,36,0,0,0,0,0,0
71326,2021,02/14/2021,0.0,loyola-maryland,1259.0,88,lafayette,1248.0,69,3.0,1.06,1,NaN,NaN,Pat,-4.34,97.0,101.3,-0.19,Pat,-8.44,103.1,111.5,0.062,214.0,253.0,72.117647,70.941176,1.176471,1.176471,0.464743,0.429341,0.303375,0.339861,0.607424,24,0,0,77.666667,76.666667,-0.933333,1.0,0.437864,0.459527,0.357791,0.347018,0.815865,36,0,0,0,0,0,0


In [133]:
list(all_games_df)

['Season',
 'date',
 'is_tourney',
 'home',
 'HTeamID',
 'hscore',
 'road',
 'RTeamID',
 'rscore',
 'line',
 'lineavg',
 'Hwin',
 'HSeed',
 'RSeed',
 'Hconf',
 'Hadjem',
 'Hadjo',
 'Hadjd',
 'Hluck',
 'Rconf',
 'Radjem',
 'Radjo',
 'Radjd',
 'Rluck',
 'HRankPOM',
 'RRankPOM',
 'HScore',
 'HOpponentScore',
 'HTOmargin',
 'HPointMargin',
 'HFG',
 'HFGopponent',
 'HFG3',
 'HFG3opponent',
 'HFT',
 'Hwins',
 'Hwins_top25',
 'Hwins_top5',
 'RScore',
 'ROpponentScore',
 'RTOmargin',
 'RPointMargin',
 'RFG',
 'RFGopponent',
 'RFG3',
 'RFG3opponent',
 'RFT',
 'Rwins',
 'Rwins_top25',
 'Rwins_top5',
 'Htourny20plus',
 'Rtourny20plus',
 'HBig4Conf',
 'RBig4Conf']

In [134]:
#Note: only fills in seeds and some games missing line
#mean imputations of variables for model
all_games_df = all_games_df.fillna(all_games_df.mean())

In [135]:
all_games_df.to_csv('all_games_df.csv', index=False)